In [1]:
import sys
sys.path.append('/workspace/evidence_retrieval/lxmert/src')
from lxrt.modeling import *
import torch
from torch.cuda.amp import GradScaler, autocast
import easydict
from collections import OrderedDict
import pickle as pkl
from torch.utils.data import DataLoader
import numpy as np
import os
from tqdm.auto import tqdm
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="2"
from nltk.translate.meteor_score import single_meteor_score
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [2]:
mode = 'newfinal'
exp_name = 'base'

In [3]:
class LXRTImageEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()

        # Obj-level image embedding layer
        self.visn_fc = VisualFeatEncoder(config)
        self.visn_cls = torch.nn.Parameter(torch.rand((1,768)))
        # Number of layers
        self.num_l_layers = VISUAL_CONFIG.l_layers
        self.num_x_layers = VISUAL_CONFIG.x_layers
        self.num_r_layers = VISUAL_CONFIG.r_layers
        print("LXRT encoder with %d l_layers, %d x_layers, and %d r_layers." %
              (self.num_l_layers, self.num_x_layers, self.num_r_layers))
        self.visn_cls = torch.nn.Parameter(torch.rand((1,1,768)) , requires_grad=True)
        # Layers
        # Using self.layer instead of self.l_layer to support loading BERT weights.
        self.r_layers = nn.ModuleList(
            [BertLayer(config) for _ in range(self.num_r_layers)]
        )

    def forward(self, lang_feats=None, lang_attention_mask=None,
                visn_feats=None, visn_attention_mask=None):
        # Run visual embedding layer
        # Note: Word embedding layer was executed outside this module.
        #       Keep this design to allow loading BERT weights.
        visn_feats = self.visn_fc(visn_feats)
        visn_cls = self.visn_cls.repeat((visn_feats.shape[0],1,1))
        visn_feats = torch.cat((visn_cls, visn_feats),dim = -2 )
        # Run cross-modality layers
        for layer_module in self.r_layers:
            visn_feats = layer_module(visn_feats, visn_attention_mask)

        return visn_feats

class BertPooler(nn.Module):
    def __init__(self, config):
        super(BertPooler, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output

class LXRTImageModel(BertPreTrainedModel):
    """LXRT Model."""

    def __init__(self, config):
        super().__init__(config)
        self.embeddings = BertEmbeddings(config)
        self.encoder = LXRTImageEncoder(config)
        self.pooler = BertPooler(config)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids=None, token_type_ids=None, attention_mask=None,
                visual_feats=None, visual_attention_mask=None):
        if visual_attention_mask is not None:
            extended_visual_attention_mask = visual_attention_mask.unsqueeze(1).unsqueeze(2)
            extended_visual_attention_mask = extended_visual_attention_mask.to(dtype=next(self.parameters()).dtype) # fp16 compatibility
            extended_visual_attention_mask = (1.0 - extended_visual_attention_mask) * -10000.0
        else:
            extended_visual_attention_mask = None
        # Run LXRT backbone
        visn_feats = self.encoder(
            None,
            None,
            visn_feats=visual_feats,
            visn_attention_mask=extended_visual_attention_mask)
        pooled_output = self.pooler(visn_feats)

        return visn_feats, pooled_output

In [18]:
import os
import pickle as pkl
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from statistics import mean
from tqdm import tqdm
import csv
import sys
import time
import numpy as np
import base64
class segmentDataset(Dataset):
    """Segment dataset."""
    def __init__(self, vidlistpkl, feat_base, cook2_IVD_dir ="/workspace/evidence_retrieval/COOK2_IVD", mode = 'train' ):
        with open(os.path.join(vidlistpkl), "rb") as f:
            vid_pkl = pkl.load(f)
            
        self.vid_list = vid_pkl
        self.feat_base = feat_base
        with open(os.path.join(cook2_IVD_dir, f"pkl/{mode}.pkl"), "rb") as f:
            self.vid_pkl = pkl.load(f)
    def __len__(self):
        return len(self.vid_list)

    def __getitem__(self, idx):
        

        vid = self.vid_list[idx]
        with open(os.path.join(self.feat_base, vid+'.pkl'), 'rb') as fp:
                  item = pkl.load(fp)
        visn_feats, encode, temporal_label, label = item
        query = self.vid_pkl[vid]["query"]
        
        
        
        return {"vid": vid, "visn_feats":visn_feats, 'encode':encode, 'temporal_label':temporal_label,'label':label, 'query':query}

In [19]:
segment_dataset = segmentDataset('newsplit_train_vid_list.pkl','feat_dump')
valid_dataset = segmentDataset('newsplit_valid_vid_list.pkl','feat_dump', mode = 'vid')
test_dataset = segmentDataset('newsplit_test_vid_list.pkl','feat_dump', mode = 'test')

In [6]:
class prf_expansion(nn.Module):
    def __init__(self, is_cuda = True, modality = 'lang'):
        super(prf_expansion, self).__init__()
        self.softmax = nn.Softmax(dim=-1)
        self.confounder_modaltiy = modality
        if confounder_modality == 'object':
            self.Z = torch.from_numpy( np.load('dic_v.npy') ).cuda().float()
            self.prior = torch.from_numpy(np.load('prior_v.npy')).cuda().float()
            self.y_att_matrix = nn.Linear(768,768)
            self.z_att_matrix = nn.Linear(2048,768) 
            self.e_z_x_transform = nn.Linear(2048,768)
        else:
            self.Z = torch.from_numpy( np.load('dic_t.npy') ).cuda().float()
            self.prior = torch.from_numpy(np.load('prior_t.npy')).cuda().float()
            self.y_att_matrix = nn.Linear(768,768)
            self.z_att_matrix = nn.Linear(768,768)
            self.e_z_x_transform = nn.Linear(768,768)
    def forward(self, y, vid = None):
            y = self.y_att_matrix(y)
            z = self.z_att_matrix(self.Z)
            a = self.softmax(torch.matmul(y, z.transpose(1,0)))
            a = self.prior * a        
            E_z_x = torch.matmul(a,self.Z)
            E_z_x = self.e_z_x_transform(E_z_x)
            return E_z_x

"class Do_Calculus(nn.Module):\n    def __init__(self, is_cuda = True, confounder_modality = 'lang'):\n        super(Do_Calculus, self).__init__()\n        with open(confounder_modality+'_confounder.pkl', 'rb') as fp:\n            self.Z = pkl.load( fp)\n        with open(confounder_modality+'_prior.pkl', 'rb') as fp:\n            self.prior = pkl.load( fp)\n        self.softmax = nn.Softmax(dim=-1)\n        self.confounder_modaltiy = confounder_modality\n        if confounder_modality == 'object':\n            for key in self.Z.keys():\n                if is_cuda:\n                    self.Z[key] = torch.from_numpy(self.Z[key]).cuda().float()\n                else:\n                    self.Z[key] = torch.from_numpy(self.Z[key]).float()\n            for key in self.prior.keys():\n                if is_cuda:\n                    self.prior[key] = torch.from_numpy(self.prior[key]).cuda().float()\n                else:\n                    self.prior[key] = torch.from_numpy(self.prior[ke

In [7]:
class expansion_merge(nn.Module):
    def __init__(self):
        super(Do_Evaluator, self).__init__()
        
        self.activation = nn.GELU()
        self.m1 = nn.Linear(768*2,768, bias = True)
        self.m2 = nn.Linear(768*2,768, bias = True)
    def forward(self, x):
        x = self.m1(x)
        #x = self.activation(x)
        #x = self.m2(x)
        return x

In [8]:
with open('yc2_recipes.json', 'r') as fp:
    recipe_dict = json.load(fp)

In [9]:
config = BertConfig('bert_config.json')
image_encoder = LXRTImageModel(config)
state_dict_path = os.path.join('lxmert', 'snap', 'pretrained', 'model_LXRT.pth') 
state_dict = torch.load(state_dict_path)
new_state_dict = OrderedDict()
for key, value in state_dict.items():
    splittedkey = key.split('.')
    if 'bert' in splittedkey:
        newkey  = '.'.join(splittedkey[splittedkey.index('bert')+1:])
    else:
        newkey  = '.'.join(splittedkey[splittedkey.index('module')+1:])
    new_state_dict[newkey] = value
image_encoder.load_state_dict(new_state_dict, strict=False)
image_encoder.cuda().train()
print()

LXRT encoder with 9 l_layers, 5 x_layers, and 5 r_layers.



In [10]:
from transformers import BertModel, BertTokenizer
from transformers.models.bert.modeling_bert import BertOnlyMLMHead, BertConfig
config = BertConfig.from_pretrained('bert-base-uncased')
lang_encoder = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
temporal_classifier = torch.nn.Linear(768*2,3).cuda().train()
lang_encoder.cuda().train()
expansion_obj = prf_expansion(modality = 'object')
expansion_obj.cuda().train()
expansion_word = prf_expansion(modality = 'lang')
expansion_word.cuda().train()
merge1 =  expansion_merge()
merge1.cuda()
merge2 =  expansion_merge()
merge2.cuda()
print()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
from lxrt.optimization import BertAdam
epoch = 100
batch_per_epoch = len(segment_dataset)
t_total = int(batch_per_epoch * epoch)
warmup_ratio = 0.05
warmup_iters = int(t_total * warmup_ratio)
optim = BertAdam(list(image_encoder.parameters()) + list(lang_encoder.parameters())+list(temporal_classifier.parameters())+list(do_cal.parameters())+list(do_evaluator.parameters())+list(do_cal2.parameters())+list(do_evaluator2.parameters()),
                       lr=1e-4, warmup=warmup_ratio, t_total=t_total)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-1)
scaler = GradScaler()

In [13]:
flatten_recipe = {}
for foodname in recipe_dict.keys():
    flatten_recipe[foodname] = []
    for item in recipe_dict[foodname]:
        flatten_recipe[foodname].extend(item['split_ins'])

In [15]:
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
writer = SummaryWriter(f'{mode}/{exp_name}_{timestamp}')
print(timestamp)

2021-12-24 08:37:13


In [16]:
exp_name

'base'

In [17]:
epochs = 100

In [ ]:
#train
min_loss = 0
sampling_num = 3
temporal_weight = 0.1
print(mode)
for epoch in tqdm(range(epochs)):
    random_idx = list(range(len(segment_dataset)))
    random.shuffle(random_idx)
    for batch_idx in range(len(segment_dataset)):
        batch = segment_dataset.__getitem__(random_idx[batch_idx])
        vid, visn_feats, encode, temporal_label, label = batch['vid'], batch["visn_feats"],batch['encode'], batch['temporal_label'],batch['label']
        frame_feats, box_feats = visn_feats
        foodname = batch['query']
        optim.zero_grad()
        image_encoder.train()
        lang_encoder.train()
        expansion_obj.train()
        expansion_word.train()
        merge1.train()
        merge2.train()
        temporal_classifier.train()
        with autocast():
            temporal_label = torch.tensor(temporal_label).long().cuda()
            label = torch.tensor(label).long().cuda()
            visn_feats = torch.tensor(frame_feats).float().cuda(), torch.tensor(box_feats).float().cuda()
            seq_output, pooled_output = image_encoder(visual_feats = visn_feats)
            encode = encode.cuda()
            output = lang_encoder(encode)
            sequence_output, lang_pooled_output = output[0], output[1]
            expanded_word = expansion_word(pooled_output, vid = vid)
            concat1 = torch.cat(( pooled_output, expanded_word), dim = -1)
            concat1 = merge1(concat1)
            expanded_obj = expansion_obj(lang_pooled_output, vid = vid)
            concat2 = torch.cat((lang_pooled_output, do_output2), dim = -1)
            concat2 = merge2(concat2)
            dotoutput = torch.matmul(concat1, concat2.transpose(1,0))
            loss = loss_fn(dotoutput, label)
            writer.add_scalar('Loss/train',loss.detach().cpu(), epoch*len(segment_dataset)+batch_idx)
            temporal_sample = []
            for visn_i in range(len(pooled_output)):
                    for lang_i in range(len(lang_pooled_output)):
                        temporal_sample.append(torch.cat((pooled_output[visn_i],
                                                          lang_pooled_output[lang_i]), -1).unsqueeze(0))
            temporal_sample = torch.cat(temporal_sample, 0)
            temporal_output = temporal_classifier(temporal_sample)
            temporal_loss = loss_fn(temporal_output, temporal_label)
            loss = loss+temporal_weight*temporal_loss
            writer.add_scalar('Loss/train_total',loss.detach().cpu(), epoch*len(segment_dataset)+batch_idx)
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(list(image_encoder.parameters()) + list(lang_encoder.parameters())+list(temporal_classifier.parameters())+list(do_cal.parameters())+list(do_evaluator.parameters()),1 )
        scaler.step(optim)
        scaler.update()
    torch.save(lang_encoder.state_dict(), os.path.join(f'{mode}','lang_encoder_epoch{}.pth'.format(epoch)))
    torch.save(image_encoder.state_dict(),  os.path.join(f'{mode}','image_encoder_epoch{}.pth'.format(epoch)))
    torch.save(expansion_obj.state_dict(), os.path.join(f'{mode}','do_cal_epoch{}.pth'.format(epoch)))
    torch.save(expansion_word.state_dict(), os.path.join(f'{mode}','do_eval_{}.pth'.format(epoch)))
    torch.save(merge1.state_dict(),os.path.join(f'{mode}','do_cal2_epoch{}.pth'.format(epoch)))
    torch.save(merge2.state_dict(), os.path.join(f'{mode}','do_eval2_{}.pth'.format(epoch)))
    with torch.no_grad():
            ks = [1,3,5]
            recalls = {k:[] for k in ks}
            image_encoder.eval()
            lang_encoder.eval()
            expansion_obj.eval()
            expansion_word.eval()
            merge1.eval()
            merge2.eval()
            temporal_classifier.eval()
            with autocast():
                losses = 0
                for batch_idx in range(len(valid_dataset)):
                    batch = valid_dataset.__getitem__(batch_idx)
                    vid, visn_feats, encode, temporal_label, label = batch['vid'], batch["visn_feats"],batch['encode'], batch['temporal_label'],batch['label']
                    frame_feats, box_feats = visn_feats
                    foodname = batch['query']
                    temporal_label = torch.tensor(temporal_label).long().cuda()
                    label = torch.tensor(label).long().cuda()
                    visn_feats = torch.tensor(frame_feats).float().cuda(), torch.tensor(box_feats).float().cuda()
                    seq_output, pooled_output = image_encoder(visual_feats = visn_feats)
                    encode = encode.cuda()
                    output = lang_encoder(encode)
                    sequence_output, lang_pooled_output = output[0], output[1]
                    expanded_word = expansion_word(pooled_output, vid = vid)
                    concat1 = torch.cat(( pooled_output, expanded_word), dim = -1)
                    concat1 = merge1(concat1)
                    expanded_obj = expansion_obj(lang_pooled_output, vid = vid)
                    concat2 = torch.cat((lang_pooled_output, do_output2), dim = -1)
                    concat2 = merge2(concat2)
                    dotoutput = torch.matmul(concat1, concat2.transpose(1,0))
                    label = list(label.detach().cpu().numpy())
                    for k in ks:
                        pred = np.argsort(-1*dotoutput.detach().cpu().numpy(), axis = -1)[:,:k].squeeze()
                        recallatk = 0
                        examples = 0
                        for i, gt in enumerate(label):
                            if k > 1:
                                if gt == -1:
                                    continue
                                if gt in pred[i]:
                                    recallatk +=1
                                examples += 1
                            else:
                                if gt == -1:
                                    continue
                                if gt == pred[i]:
                                    recallatk +=1
                                examples += 1
                        recallatk = recallatk/examples
                        recalls[k].append(recallatk)
                for k in ks:
                    print(epoch, k, sum(recalls[k])/len(recalls[k]))
                    writer.add_scalar(f'Recall/{k}',sum(recalls[k])/len(recalls[k]), epoch)


  0%|          | 0/100 [00:00<?, ?it/s]

newfinal


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/workspace/evidence_retrieval/lxmert/src/lxrt/optimization.py:142: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tens

0 1 0.19477656702470966
0 3 0.5197850611254626
0 5 0.7503630852031805


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:144: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:145: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  1%|          | 1/100 [03:30<5:46:47, 210.18s/it]

...

58 1 0.3820391848345664
58 3 0.7061899792786739
58 5 0.8635434583280149
59 1 0.373988329955571
59 3 0.7046222534040003
59 5 0.8692747959744046


 60%|██████    | 60/100 [3:27:23<2:16:56, 205.40s/it]

59 1 0.37780542004393264
59 3 0.7011760785690387
59 5 0.861968357859748
60 1 0.36981083152711064
60 3 0.7023295008656388
60 5 0.8685180432809667


 61%|██████    | 61/100 [3:30:49<2:13:33, 205.48s/it]

60 1 0.3723090105334006
60 3 0.6970740421225579
60 5 0.865022661219864


In [16]:
#load trained model
epoch = 23
lang_encoder.load_state_dict(torch.load(os.path.join(f'{mode}','lang_encoder_epoch{}.pth'.format(epoch))))
image_encoder.load_state_dict(torch.load(  os.path.join(f'{mode}','image_encoder_epoch{}.pth'.format(epoch))))
expansion_obj.load_state_dict(torch.load(os.path.join(f'{mode}','expansion_obj_epoch{}.pth'.format(epoch))))
expansion_word.load_state_dict(torch.load( os.path.join(f'{mode}','expansion_word_{}.pth'.format(epoch))))
merge1.load_state_dict(torch.load(os.path.join(f'{mode}','merge1_epoch{}.pth'.format(epoch))))
merge2.load_state_dict(torch.load( os.path.join(f'{mode}','merge2_{}.pth'.format(epoch))))

<All keys matched successfully>

In [20]:
    #eval recall on test set
    with torch.no_grad():
            ks = [1,3,5]
            recalls = {k:[] for k in ks}
            image_encoder.eval()
            lang_encoder.eval()
            expansion_obj.eval()
            expansion_word.eval()
            merge1.eval()
            merge2.eval()
            with autocast():
                losses = 0
                for batch_idx in range(len(test_dataset)):
                    batch = valid_dataset.__getitem__(batch_idx)
                    vid, visn_feats, encode, temporal_label, label = batch['vid'], batch["visn_feats"],batch['encode'], batch['temporal_label'],batch['label']
                    frame_feats, box_feats = visn_feats
                    foodname = batch['query']
                    temporal_label = torch.tensor(temporal_label).long().cuda()
                    label = torch.tensor(label).long().cuda()
                    visn_feats = torch.tensor(frame_feats).float().cuda(), torch.tensor(box_feats).float().cuda()
                    seq_output, pooled_output = image_encoder(visual_feats = visn_feats)
                    encode = encode.cuda()
                    output = lang_encoder(encode)
                    sequence_output, lang_pooled_output = output[0], output[1]
                    expanded_word = expansion_word(pooled_output, vid = vid)
                    concat1 = torch.cat(( pooled_output, expanded_word), dim = -1)
                    concat1 = merge1(concat1)
                    expanded_obj = expansion_obj(lang_pooled_output, vid = vid)
                    concat2 = torch.cat((lang_pooled_output, do_output2), dim = -1)
                    concat2 = merge2(concat2)
                    dotoutput = torch.matmul(concat1, concat2.transpose(1,0))
                    label = list(label.detach().cpu().numpy())
                    for k in ks:
                        pred = np.argsort(-1*dotoutput.detach().cpu().numpy(), axis = -1)[:,:k].squeeze()
                        recallatk = 0
                        examples = 0
                        for i, gt in enumerate(label):
                            if k > 1:
                                if gt == -1:
                                    continue
                                if gt in pred[i]:
                                    recallatk +=1
                                examples += 1
                            else:
                                if gt == -1:
                                    continue
                                if gt == pred[i]:
                                    recallatk +=1
                                examples += 1
                        recallatk = recallatk/examples
                        recalls[k].append(recallatk)
                for k in ks:
                    print(epoch, k, sum(recalls[k])/len(recalls[k]))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


23 1 0.39642943092929456
23 3 0.7226285046305423
23 5 0.8752401518522226


In [21]:
import os
import pickle as pkl
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from statistics import mean
from tqdm import tqdm
import csv
import sys
import time
import numpy as np
import base64
cook2_IVD_dir = "/workspace/evidence_retrieval/COOK2_IVD"


with open(os.path.join(cook2_IVD_dir, "pkl/{}.pkl".format("train")), "rb") as f:
    vid_pkl = pkl.load(f)
    
csv.field_size_limit(sys.maxsize)
FIELDNAMES = ["img_id", "img_h", "img_w", "objects_id", "objects_conf",
              "attrs_id", "attrs_conf", "num_boxes", "boxes", "features"]


def load_obj_tsv(fname, topk=None):
    """Load object features from tsv file.
    :param fname: The path to the tsv file.
    :param topk: Only load features for top K images (lines) in the tsv file.
        Will load all the features if topk is either -1 or None.
    :return: A list of image object features where each feature is a dict.
        See FILENAMES above for the keys in the feature dict.
    """
    data = []
    start_time = time.time()
    #print("Start to load Faster-RCNN detected objects from %s" % fname)
    with open(fname) as f:
        reader = csv.DictReader(f, FIELDNAMES, delimiter="\t")
        for i, item in enumerate(reader):

            for key in ['img_h', 'img_w', 'num_boxes']:
                item[key] = int(item[key])
            
            boxes = item['num_boxes']
            decode_config = [
                ('objects_id', (boxes, ), np.int64),
                ('objects_conf', (boxes, ), np.float32),
                ('attrs_id', (boxes, ), np.int64),
                ('attrs_conf', (boxes, ), np.float32),
                ('boxes', (boxes, 4), np.float32),
                ('features', (boxes, -1), np.float32),
            ]
            for key, shape, dtype in decode_config:
                item[key] = np.frombuffer(base64.b64decode(item[key]), dtype=dtype)
                item[key] = item[key].reshape(shape)
                item[key].setflags(write=False)

            data.append(item)
            if topk is not None and len(data) == topk:
                break
    elapsed_time = time.time() - start_time
    #print("Loaded %d images in file %s in %d seconds." % (len(data), fname, elapsed_time))
    return data

class segmentDataset(Dataset):
    """Segment dataset."""
    def __init__(self, cook2_IVD_dir, mode = "train"):
        with open(os.path.join(cook2_IVD_dir, "pkl/{}.pkl".format(mode)), "rb") as f:
            vid_pkl = pkl.load(f)
        
        self.vid_list = list(vid_pkl.keys())
        self.vid_pkl = vid_pkl

        self.max_frame_len = 552 ### IF you change this, segment_collate also needs to change !!!!!!
    def __len__(self):
        return len(self.vid_pkl)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        vid = self.vid_list[idx]
        max_frame_len = self.max_frame_len
        
        index_transcript = self.vid_pkl[vid]["index_trainscirpt"]
        segments_ = self.vid_pkl[vid]["segments"]
        
        trans_idx = torch.tensor([idx for idx, trans in index_transcript])
        trans = [trans for idx, trans in index_transcript]
        
        segs_idx = torch.tensor([(idx_start, idx_end) for idx_start, idx_end, segs in segments_])
        segs = [segs for idx_start, idx_end, segs in segments_]
        
        begin_idxs = list( int(b.cpu().tolist()) for b, e in segs_idx)
        end_idxs = list( int(e.cpu().tolist()) for b, e in segs_idx)

        
        query = self.vid_pkl[vid]["query"]
        frame_info = load_obj_tsv(os.path.join(cook2_IVD_dir, "features/{}_obj36.tsv".format(vid)))
        
        ## Make beign target  ( EX) [0, 0, 0, 1, ..... ,0, 1, 0, 0]
        begin_target = []
        end_target = []
        
        begin_distance = []
        end_distance = []
        
        begin_ratio = []
        end_ratio = []
        
        
        mask = []
        
        for idx in range(max_frame_len):
            __flag__ = 0
            target_b = -1
            target_e = -1
            middle = -1
            for b, e in segs_idx:
                if(idx >= b and idx <= e): __flag__ = 1; target_b = b; target_e = e; middle = (b + e) / 2.0
                    
            
            if(__flag__): 
                begin_target.append(1.); end_target.append(1.);
                begin_distance.append(abs(target_b - idx)); end_distance.append(abs(target_e - idx)) 
                begin_ratio.append(torch.sigmoid(middle - idx)); end_ratio.append(torch.sigmoid(idx - middle))
                
                
            else: 
                begin_target.append(0.); end_target.append(0.)
                begin_distance.append(999.); end_distance.append(999.)
                begin_ratio.append(0.); end_ratio.append(0.)
                
        
            if(idx < len(frame_info)): 
                mask.append(True)
            else: 
                mask.append(False)
        """
        for idx in range(max_frame_len):
            if(idx in begin_idxs): begin_target.append(1.)
            else: begin_target.append(0.)
                
            if(idx in end_idxs): end_target.append(1.)
            else: end_target.append(0.)
                
        
            if(idx < len(frame_info)): mask.append(True)
            else: mask.append(False)
        """

        
        
        
        return {"vid": vid, "trans_idx": trans_idx, "trans": trans, "segs_idx": segs_idx, "segs": segs, "query": query, "frame_info": frame_info, \
               "begin_target": begin_target, "end_target": end_target, "mask": mask,\
               "begin_distance": begin_distance, "end_distance": end_distance, "begin_ratio": begin_ratio, "end_ratio": end_ratio}
    
    
def segment_collate(samples):
    vid, query = [], []
    trans_idx, trans = [], []
    segs_idx, segs = [], []
    frame_info = []
    
    begin_target, end_target = [], []
    begin_distance, end_distance =  [], []
    begin_ratio, end_ratio = [], []
    mask = []
    
    max_trans_len = max( list(len(s["trans_idx"]) for s in samples))
    max_segs_len = max( list(len(s["segs_idx"]) for s in samples))
    #max_frame_len = max(list(len(s["frame_info"]) for s in samples))
    max_frame_len = 552

    for sample in samples:
        vid.append(sample["vid"])
        query.append(sample["query"])
        
        trans_idx.append(sample["trans_idx"])
        trans.append(sample["trans"] + ["<PAD>"] * (max_trans_len - len(sample["trans"])) )
        
        segs_idx.append(sample["segs_idx"])
        segs.append(sample["segs"] + ["<PAD>"] * (max_segs_len - len(sample["segs"])) )
        begin_target.append(sample["begin_target"])
        end_target.append(sample["end_target"])
        
        begin_distance.append(sample["begin_distance"]); end_distance.append(sample["end_distance"])
        begin_ratio.append(sample["begin_ratio"]); end_ratio.append(sample["end_ratio"])
        mask.append(sample["mask"])
        
        frame_info.append(sample["frame_info"] + ["<PAD>"] * (max_frame_len - len(sample["frame_info"])))
        
    padded_trans_idx = torch.nn.utils.rnn.pad_sequence(trans_idx, batch_first=True, padding_value = -4444).contiguous()
    padded_trans = trans 
    padded_segs_idx = torch.nn.utils.rnn.pad_sequence(segs_idx, batch_first=True, padding_value = -4444).contiguous()
    padded_segs = segs 


    return {"vid": vid, "trans_idx": padded_trans_idx, "trans": padded_trans, \
            "segs_idx": padded_segs_idx, "segs": padded_segs, "query": query, "frame_info": frame_info,\
           "begin_target": torch.tensor(begin_target), "end_target": torch.tensor(end_target), "mask": torch.tensor(mask),\
           "begin_distance": begin_distance, "end_distance": end_distance, "begin_ratio": begin_ratio, "end_ratio": end_ratio}

In [22]:

segment_dataset = segmentDataset(cook2_IVD_dir)
valid_dataset = segmentDataset(cook2_IVD_dir, mode = 'test')


In [23]:
dataloader = DataLoader(segment_dataset, collate_fn = segment_collate, batch_size = 1, shuffle = True, num_workers = 4 )
validloader = DataLoader(valid_dataset, collate_fn = segment_collate, batch_size = 1, num_workers = 4)

In [24]:
with open('yc2_recipes.json', 'r') as fp:
    recipes = json.load(fp)

In [25]:
    sampling_num = 3
    matched_recipe = {}
    with torch.no_grad():
        image_encoder.eval()
        lang_encoder.eval()
        with autocast():
            losses = 0
            for i, batch in tqdm(enumerate(dataloader), total = len(dataloader)):
                maxlen = max([len(i) for i in batch['frame_info']])
                annot, segs_idx, item =batch['segs'][0],batch['segs_idx'][0],batch['frame_info'][0]
                query = batch['query'][0]
                vid = batch['vid'][0]
                vid_recipe = recipes[query]
                flatten_recipe = []
                for recipe in vid_recipe:
                    for recipe_doc in recipe['split_ins']:
                        if recipe_doc!= '':
                            flatten_recipe.append(recipe_doc)
                frame_feats = []
                box_feats = []
                frame_indices = []
                for frame_idx in range(len(item)//sampling_num):
                    frame_idx = sampling_num*frame_idx+1
                    if item[frame_idx] != '<PAD>':
                        frame_feats.append(item[frame_idx]['features'])
                        box_feats.append(item[frame_idx]['boxes'])
                        frame_indices.append(frame_idx)
                    else:
                        break
                visn_feats = torch.tensor(frame_feats).float().cuda(), torch.tensor(box_feats).float().cuda()
                seq_output, pooled_output = image_encoder(visual_feats = visn_feats)
                encode = [tokenizer.encode(sent) for sent in flatten_recipe]
                maxlen = max([len(e) for e in encode])
                encode = torch.tensor([e+[tokenizer.pad_token_id]*(maxlen-len(e)) for e in encode]).cuda()
                output = lang_encoder(encode)
                sequence_output, lang_pooled_output = output[0], output[1]
                expanded_word = expansion_word(pooled_output, vid = vid)
                concat1 = torch.cat(( pooled_output, expanded_word), dim = -1)
                concat1 = merge1(concat1)
                expanded_obj = expansion_obj(lang_pooled_output, vid = vid)
                concat2 = torch.cat((lang_pooled_output, do_output2), dim = -1)
                concat2 = merge2(concat2)
                dotoutput = torch.matmul(concat1, concat2.transpose(1,0))
                pred = np.argmax(-1*dotoutput.detach().cpu().numpy(), axis = -1)
                vid_matched_recipes = {frame_indices[j]:flatten_recipe[pred[j]] for j in range(len(pred))}
                matched_recipe[vid] = vid_matched_recipes
            for i, batch in tqdm(enumerate(validloader), total = len(validloader)):
                maxlen = max([len(i) for i in batch['frame_info']])
                annot, segs_idx, item =batch['segs'][0],batch['segs_idx'][0],batch['frame_info'][0]
                query = batch['query'][0]
                vid = batch['vid'][0]
                vid_recipe = recipes[query]
                flatten_recipe = []
                for recipe in vid_recipe:
                    for recipe_doc in recipe['split_ins']:
                        if recipe_doc!= '':
                            flatten_recipe.append(recipe_doc)
                frame_feats = []
                box_feats = []
                frame_indices = []
                for frame_idx in range(len(item)//sampling_num):
                    frame_idx = sampling_num*frame_idx+1
                    if item[frame_idx] != '<PAD>':
                        frame_feats.append(item[frame_idx]['features'])
                        box_feats.append(item[frame_idx]['boxes'])
                        frame_indices.append(frame_idx)
                    else:
                        break
                visn_feats = torch.tensor(frame_feats).float().cuda(), torch.tensor(box_feats).float().cuda()
                seq_output, pooled_output = image_encoder(visual_feats = visn_feats)
                encode = [tokenizer.encode(sent) for sent in flatten_recipe]
                maxlen = max([len(e) for e in encode])
                encode = torch.tensor([e+[tokenizer.pad_token_id]*(maxlen-len(e)) for e in encode]).cuda()
                output = lang_encoder(encode)
                sequence_output, lang_pooled_output = output[0], output[1]
                expanded_word = expansion_word(pooled_output, vid = vid)
                concat1 = torch.cat(( pooled_output, expanded_word), dim = -1)
                concat1 = merge1(concat1)
                expanded_obj = expansion_obj(lang_pooled_output, vid = vid)
                concat2 = torch.cat((lang_pooled_output, do_output2), dim = -1)
                concat2 = merge2(concat2)
                dotoutput = torch.matmul(concat1, concat2.transpose(1,0))
                pred = np.argmax(-1*dotoutput.detach().cpu().numpy(), axis = -1)
                vid_matched_recipes = {frame_indices[j]:flatten_recipe[pred[j]] for j in range(len(pred))}
                matched_recipe[vid] = vid_matched_recipes

100%|██████████| 267/267 [04:26<00:00,  1.00it/s]


In [26]:
with open('dual_encoder_final_matched_result.json', 'w') as fp:
    json.dump(matched_recipe, fp)

In [28]:
with open('dual_encoder_final_matched_result.pkl', 'wb') as fp:
    pkl.dump(matched_recipe, fp)